In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv
import os

# Load environment variables
load_dotenv()
client = OpenAI(api_key= os.getenv('openai_api'))

/Users/brianlivian/Documents/GitHub/poshmark-rag/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [7]:
URL = "https://poshmark.com/category/Men-Suits_&_Blazers-Suits?sort_by=added_desc&brand%5B%5D=All%20Saints&brand%5B%5D=Andrew%20Marc&brand%5B%5D=Armani%20Collezioni&brand%5B%5D=Banana%20Republic&brand%5B%5D=Barneys%20New%20York&brand%5B%5D=Boglioli&brand%5B%5D=Bonobos&brand%5B%5D=Brooks%20Brothers&brand%5B%5D=Burberry&brand%5B%5D=Canali&brand%5B%5D=Charles%20Tyrwhitt&brand%5B%5D=Club%20Monaco&brand%5B%5D=Corneliani&brand%5B%5D=Eleventy&brand%5B%5D=Emporio%20Armani&brand%5B%5D=Ermenegildo%20Zegna&brand%5B%5D=Etro&brand%5B%5D=Gucci&brand%5B%5D=Hart%20Schaffner%20Marx&brand%5B%5D=Hickey%20Freeman&brand%5B%5D=Hugo%20Boss&brand%5B%5D=Isaia&brand%5B%5D=Jack%20Victor&brand%5B%5D=John%20Varvatos&brand%5B%5D=Joseph%20Abboud&brand%5B%5D=Kiton&brand%5B%5D=Lardini&brand%5B%5D=Lubiam&brand%5B%5D=Luciano%20Barbera&brand%5B%5D=Massimo%20Dutti&brand%5B%5D=Pal%20Zileri&brand%5B%5D=Paul%20Smith&brand%5B%5D=Paul%20Stuart&brand%5B%5D=Ralph%20Lauren&brand%5B%5D=Ralph%20Lauren%20Black%20Label&brand%5B%5D=Ralph%20Lauren%20Blue%20Label&brand%5B%5D=Ralph%20Lauren%20Double%20RL&brand%5B%5D=Ralph%20Lauren%20Purple%20Label&brand%5B%5D=Reiss&brand%5B%5D=Robert%20Talbott&brand%5B%5D=Sandro&brand%5B%5D=SuitSupply&brand%5B%5D=Ted%20Baker&brand%5B%5D=Theory&brand%5B%5D=Topman%22&size%5B%5D=44R&size%5B%5D=43R&size%5B%5D=42R"

def get_listing_info_with_selenium(url, max_items=10):
    chrome_options = Options()
    # chrome_options.add_argument("--headless=new")
    chrome_options.add_argument("--disable-gpu")
    chrome_options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    WebDriverWait(driver, 15).until(EC.presence_of_element_located((By.TAG_NAME, "img")))

    scroll_pause_time = 2
    last_height = driver.execute_script("return document.body.scrollHeight")

    for _ in range(10):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause_time)
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    listings = driver.find_elements(By.CSS_SELECTOR, 'div[data-et-name="listing"]')
    listing_info = []

    for card in listings:
        try:
            img = card.find_element(By.TAG_NAME, "img")
            src = img.get_attribute("src")

            anchor = card.find_element(By.TAG_NAME, "a")
            listing_url = anchor.get_attribute("href")

            text_elements = card.find_elements(By.TAG_NAME, "div")
            title = ""
            price = ""
            for el in text_elements:
                txt = el.text.strip()
                if "$" in txt and not price:
                    price = txt
                elif txt and not title:
                    title = txt
            title = title.split('\n')[0]
            price = price.split('\n')[1].split(' ')[0]

            listing_info.append({
                "image_url": src,
                "title": title,
                "price": price,
                "url": listing_url
            })

        except Exception as e:
            print("Error extracting listing:", e)
            continue

        if len(listing_info) >= max_items:
            break

    driver.quit()
    return listing_info


def check_suit_appropriateness(image_url, brand, price):
    # print(brand, price, 'Brian Rocks')
    prompt = (
        f"This is a listing for a men's clothing item from the brand '{brand}', priced at '{price}'. "
        f"The brand is already known to be high-quality and fashionable. "
        f"Please analyze whether the item looks modern, stylish, and sharp — something a young professional in his 20s or 30s would wear "
        f"to a social event, date, work setting, or while going out. It should have a good fit (not overly baggy or outdated), be in good condition, and appear well-made. "
        f"Also determine whether the listed price is a good deal (e.g., a designer or great-looking item for under $150). "
        f"Only answer 'yes' if the clothing looks stylish, appropriate for a modern young professional, and priced well. Otherwise, answer 'no'."
    )





    response = client.chat.completions.create(
        model="gpt-4o-2024-08-06",
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": image_url}}
                ]
            }
        ],
        tools=[
            {
                "type": "function",
                "function": {
                    "name": "synagogue_suit_check",
                    "description": "Returns 'yes' only if the suit is from a luxury brand, is very stylish and appropriate for synagogue, and is being sold at a low price.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "answer": {
                                "type": "string",
                                "enum": ["yes", "no"]
                            }
                        },
                        "required": ["answer"]
                    }
                }
            }
        ],
        tool_choice={"type": "function", "function": {"name": "synagogue_suit_check"}},
        max_tokens=500
    )

    tool_call = response.choices[0].message.tool_calls[0]
    result = json.loads(tool_call.function.arguments)
    return result["answer"]


data = pd.DataFrame(columns = ['Listing Title', 'Price', 'URL', 'Image', 'AI Verdict'])

# --- Main Run ---
if __name__ == "__main__":
    print("Scraping listings...")
    listings = get_listing_info_with_selenium(URL, max_items=10)
    print(f"\n✅ Found {len(listings)} listings. Evaluating...\n")

    for i, listing in enumerate(listings):
        try:
            df = pd.DataFrame()
            answer = check_suit_appropriateness(
                listing["image_url"],
                brand=listing["title"],
                price=listing["price"]
            )
            df['Listing Title'] = [listing['title']]
            df ['Price'] = [listing['price']]
            df['Image'] = [listing['image_url']]
            df['URL'] = [listing['url']]
            df['AI Verdict'] =[ answer.upper()]
            data = pd.concat([data,df])

            time.sleep(1)
        except Exception as e:
            print(f"[{i+1}] ERROR – {e}")



Scraping listings...
Error extracting listing: list index out of range

✅ Found 10 listings. Evaluating...



In [8]:
pd.set_option('display.max_colwidth', None)
data

,Listing Title,Price,URL,Image,AI Verdict
0,Ermenegildo Zegna Suit 2 Pc 42S 34x28 Full Suit Blue Micro Check 100% Wool,$125,https://poshmark.com/listing/Ermenegildo-Zegna-Suit-2-Pc-42S-34x28-Full-Suit-Blue-Micro-Check-100-Wool-68003f0e74cb47ddb7288714,https://di2ponv0v5otw.cloudfront.net/posts/2025/04/16/68003f0e74cb47ddb7288714/s_68003f0f2fa0540577c4a522.jpg,YES
0,Brooks Brothers Madison BrooksCool Full Canvas Gray Striped 2-Btn Wool Suit 43R,$195,https://poshmark.com/listing/Brooks-Brothers-Madison-BrooksCool-Full-Canvas-Gray-Striped-2Btn-Wool-Suit-43R-68003d6bb9bd74e323ea2a87,https://di2ponv0v5otw.cloudfront.net/posts/2025/04/16/68003d6bb9bd74e323ea2a87/s_68003d6c2c9e44b8c7b26352.jpeg,NO
0,Gianluca Isaia Men' Sciammeria Spider Wool Suit Single Breasted Gray Cuffed Hems,$395,https://poshmark.com/listing/Gianluca-Isaia-Men-Sciammeria-Spider-Wool-Suit-Single-Breasted-Gray-Cuffed-Hems-6800379860beaf481bddbdc7,https://di2ponv0v5otw.cloudfront.net/posts/2025/04/16/6800379860beaf481bddbdc7/s_6800379ab7f0ac7cd4044a15.jpeg,NO
0,Pal Zileri Pintripe Suit 2pc Made in Italy Navy Blue US42R / EU52R 36x28,$116,https://poshmark.com/listing/Pal-Zileri-Pintripe-Suit-2pc-Made-in-Italy-Navy-Blue-US42R-EU52R-36x28-680027afb3ef62e4836e6ba4,https://di2ponv0v5otw.cloudfront.net/posts/2025/04/16/680027afb3ef62e4836e6ba4/s_680027b09936d61ef04486f9.jpeg,YES
0,Banana Republic Blazer Mens 44R Slim Fit Brown Herringbone Wool,$60,https://poshmark.com/listing/Banana-Republic-Blazer-Mens-44R-Slim-Fit-Brown-Herringbone-Wool-68000b0185c0164bdf63f0fd,https://di2ponv0v5otw.cloudfront.net/posts/2025/04/16/68000b0185c0164bdf63f0fd/s_68000b0185c0164bdf63f0fe.jpg,NO
0,Boss Hugo Boss Plaid Wool 2 Piece Suit Stretch Tailoring Gray Black Size 44R,$120,https://poshmark.com/listing/Boss-Hugo-Boss-Plaid-Wool-2-Piece-Suit-Stretch-Tailoring-Gray-Black-Size-44R-67fffa7de0f2cee417548d42,https://di2ponv0v5otw.cloudfront.net/posts/2025/04/16/67fffa7de0f2cee417548d42/s_67fffa7de0f2cee417548d43.jpg,NO
0,Brooks Brothers 1818 Regent Reda Blue Blazer Made in Italy 44 Regular W39,$46,https://poshmark.com/listing/Brooks-Brothers-1818-Regent-Reda-Blue-Blazer-Made-in-Italy-44-Regular-W39-67fff3c24fab7bd0bc7cc381,https://di2ponv0v5otw.cloudfront.net/posts/2025/04/16/67fff3c24fab7bd0bc7cc381/s_67fff3c385001210dc16c961.jpg,YES
0,BANANA REPUBLIC Men's Gray Double-Breasted Blazer,$170,https://poshmark.com/listing/BANANA-REPUBLIC-Mens-Gray-DoubleBreasted-Blazer-67ffeb15b7f0acd49cf86511,https://di2ponv0v5otw.cloudfront.net/posts/2025/04/16/67ffeb15b7f0acd49cf86511/s_67ffeb4b5919e0330daad6ac.jpg,NO
0,Armani Collezioni G-Line 44R 36x32 Dark Ink Blue 2 Piece Suit Virgin Wool Modern,$445,https://poshmark.com/listing/Armani-Collezioni-GLine-44R-36x32-Dark-Ink-Blue-2-Piece-Suit-Virgin-Wool-Modern-67ffeac534e98435dba70e4a,https://di2ponv0v5otw.cloudfront.net/posts/2025/04/16/67ffeac534e98435dba70e4a/s_67ffeac6b09b7c536063fbdd.jpeg,NO
0,Hugo Boss Gable/Vegas Suit Striped Guabello Luxair Italy Wool Mohair 42R 34x30,$100,https://poshmark.com/listing/Hugo-Boss-GableVegas-Suit-Striped-Guabello-Luxair-Italy-Wool-Mohair-42R-34x30-67ffb309af7f4711e3fc14fd,https://di2ponv0v5otw.cloudfront.net/posts/2025/04/16/67ffb309af7f4711e3fc14fd/s_67ffb30d1493770b0823d7e0.jpeg,YES
